
# Packing CAT Cell demo notebook
The main functions of this demo are located in [chordal_axis_transform.py](../irregular_object_packing/packing/chordal_axis_transform.py)


### Contents
1. Simple CAT Cell demo
2. Larger Demo with cylindrical container and multiple blood cell meshes.

### Pre-requisites
Please install the required packages using the following command:

```bash
pip install -r requirements.txt
```




## Basic CAT Cell demo
The cell below shows a simple demo of computing cat faces. The container is a cube and the object os a cuboid rotated 45 degrees over the z axis. The red faces are the CAT faces that belong to the cube in the back. The yellow arrows are the normal vectors, which point inwards, of each face. the yellow faces are the cat faces that belong to the other objects.

In [1]:
import numpy as np
import pyvista as pv
RES_FOLDER = "renders"    
pv.set_plot_theme("document")
def check_input_and_tetmesh_consistency(tetmesh, meshes):
    # assert tetmesh.n_points == sum([mesh.n_points for mesh in meshes]), "number of points in tetmesh and meshes do not match
    start_idx = 0
    for i, mesh in enumerate(meshes):
        end_idx = start_idx + mesh.n_points
        try:
            np.testing.assert_array_equal(tetmesh.points[start_idx:end_idx], mesh.points)
        except AssertionError as e:
            print(f"mesh {i} does not match tetmesh {e}")
        start_idx = end_idx


def compute_steiner_points(tetmesh, meshes):
    steiner_points = tetmesh.points[range(tetmesh.n_points - sum([mesh.n_points for mesh in meshes]))]
    print(f"N steiner points: {len(steiner_points)}")
    return steiner_points

def print_cat_analysis(cat_meshes):
    for i, cat in enumerate(cat_meshes):
        print(f"cat {i}: {cat.n_points}, is manifold: {cat.is_manifold}")


## 4 Cubes

In [2]:
from irregular_object_packing.cat.chordal_axis_transform import compute_cdt, compute_cat_faces, filter_relevant_cells


from irregular_object_packing.cat.utils import get_tetmesh_cell_arrays

container = pv.Cube(center=(0, 0, 0), x_length=4, y_length=4, z_length=3).triangulate().clean()

cube4 = pv.Cube(center=(-1, -1, 0), x_length=1, y_length=1, z_length=1).triangulate().clean()
cube1 = pv.Cube(center=(1, 1, 0), x_length=1, y_length=1, z_length=1).triangulate().clean()
cube2 = pv.Cube(center=(1, -1, 0), x_length=1, y_length=1, z_length=1).triangulate().clean()
cube3 = pv.Cube(center=(-1, 1, 0), x_length=1, y_length=1, z_length=1).triangulate().clean()
cube5 = pv.Cube(center=(0, 0, 0), x_length=0.9, y_length=2, z_length=1).triangulate().clean()

meshes = [cube1, cube2, cube3, cube4, container]#, cube5]
tetmesh = compute_cdt(meshes)

tet_cells = get_tetmesh_cell_arrays(tetmesh)
npoints_for_meshes = [mesh.n_points for mesh in meshes]
npoints_for_meshes[-1] += compute_steiner_points(tetmesh, meshes).shape[0]
relevant_cells, filtered_out = filter_relevant_cells(tet_cells, npoints_for_meshes)
assert len(filtered_out) != 0

plotter = pv.Plotter()
for mesh in meshes[:-1]:
    plotter.add_mesh(mesh, color="red", opacity=0.5, show_edges=True)
plotter.add_points(tetmesh.points, color="black", opacity=1, show_vertices=True)
plotter.add_mesh(meshes[-1], color="white", opacity=0.5, show_edges=True)



plotter.show()


N steiner points: 0


Widget(value="<iframe src='http://localhost:63060/index.html?ui=P_0x15547bac0_0&reconnect=auto' style='width: …

In [3]:

from irregular_object_packing.cat.chordal_axis_transform import process_cells_to_normals
from irregular_object_packing.cat.utils import get_cell_arrays
from irregular_object_packing.mesh.utils import convert_faces_to_polydata_input

steiner_points = compute_steiner_points(tetmesh, meshes)
n_points_per_obj = [len(mesh.points) for mesh in meshes] + [len(container.points)+ len(steiner_points)]
# face_normals, cat_cells, face_normals_pp = compute_cat_faces(tetmesh, n_points_per_obj, [])
cells = get_cell_arrays(tetmesh.cells)

rel_cells, _ = filter_relevant_cells(cells, n_points_per_obj)

face_normals, cat_cells, normalspp = process_cells_to_normals(tetmesh.points, rel_cells, len(n_points_per_obj))

cat_meshes = []
for cat_cell in cat_cells[:-1]:
    cat_meshes.append(pv.PolyData(*convert_faces_to_polydata_input(cat_cell)))
    

print_cat_analysis(cat_meshes)
check_input_and_tetmesh_consistency(tetmesh, meshes+ [container])

plotter = pv.Plotter()
# plotter.add_mesh_clip_plane(tetmesh, color="grey", show_edges=True, opacity=0.9, crinkle=True)
for mesh in meshes:
    plotter.add_mesh(mesh, color="red", show_edges=True, opacity=1,)

plotter.show()

N steiner points: 0
cat 0: 78, is manifold: True
cat 1: 58, is manifold: True
cat 2: 58, is manifold: True
cat 3: 62, is manifold: True
cat 4: 108, is manifold: True
mesh 5 does not match tetmesh 
Arrays are not equal

(shapes (0, 3), (8, 3) mismatch)
 x: pyvista_ndarray([], shape=(0, 3), dtype=float64)
 y: pyvista_ndarray([[-2. , -2. , -1.5],
                 [-2. , -2. ,  1.5],
                 [-2. ,  2. , -1.5],...


Widget(value="<iframe src='http://localhost:63060/index.html?ui=P_0x1663d4d30_1&reconnect=auto' style='width: …

In [4]:
from irregular_object_packing.mesh.collision import compute_collision


cell_id = 3
def plot_single_mesh_with_cat(cell_id):
    plotter = pv.Plotter()
    plotter.add_mesh(meshes[cell_id], color="red", show_edges=True, opacity=1,)
    plotter.add_mesh(cat_meshes[cell_id], color="yellow", show_edges=True, opacity=0.5,)
    plotter.show()

    plotter.add_mesh(container, color="white", show_edges=True, opacity=0.2,)
    for i, mesh in enumerate(meshes[:-1]):
        if i != cell_id:
            plotter.add_mesh(mesh, color="red", show_edges=True, opacity=0.2,)
    return plotter

pltr = plot_single_mesh_with_cat(cell_id)
plotter.show()
pltr.save_graphic(f"{RES_FOLDER}/4cubes-6-result.pdf")


Widget(value="<iframe src='http://localhost:63060/index.html?ui=P_0x1554af9d0_2&reconnect=auto' style='width: …

A view with name (P_0x1663d4d30_1) is already registered
 => returning previous one


Widget(value="<iframe src='http://localhost:63060/index.html?ui=P_0x1663d4d30_1&reconnect=auto' style='width: …

In [5]:

def sort_cell_ids_by_z_x(cell_centers, cell_ids):
    # sort ids based on z, x of center of cell

    sorted_cells = sorted(zip(cell_ids, cell_centers.points), key=lambda x: (x[1][2], x[1][0], x[1][1]))
    sorted_cell_ids = [x[0] for x in sorted_cells]
    return sorted_cell_ids

In [6]:
file_name = f"{RES_FOLDER}/4cubes_1_orbit_start"
plotter = pv.Plotter(off_screen=True)
p_c = plotter.add_mesh(container, color="white", show_edges=True, opacity=0.2,)
actors = []
for mesh in meshes[:-1]:
    actors.append(plotter.add_mesh(mesh, color="red", show_edges=True, opacity=1,))
# plotter.add_mesh(cat_meshes[cell_id], color="yellow", show_edges=True, opacity=0.5,)
path = plotter.generate_orbital_path(n_points=36,viewup=(0,0,1),)
plotter.open_gif(f"{file_name}.gif")
plotter.orbit_on_path(path, write_frames=True)
plotter.close()


In [7]:
file_name = f"{RES_FOLDER}/4cubes_2_points"
plotter = pv.Plotter(off_screen=True, notebook=False)
plotter.open_gif(f"{file_name}.gif",)
plotter.add_mesh(container, color="white", show_edges=True, opacity=0.2,)
for mesh in meshes[:-1]:
    plotter.add_mesh(mesh, color="red", show_edges=True, opacity=0.5,)

plotter.write_frame()
plotter.save_graphic(f"{file_name}-pre.pdf")
for mesh in meshes:
    # add only the points of the meshes
    plotter.add_points(mesh.points, color="black", opacity=1, render_points_as_spheres=True, point_size=10)
    plotter.write_frame()
    
plotter.save_graphic(f"{file_name}-post.pdf")
plotter.close()

In [8]:
file_name = f"{RES_FOLDER}/4cubes_3_orbit_tets"
plotter = pv.Plotter(off_screen=True, notebook=False)
plotter.open_gif(f"{file_name}.gif",)

# recreate previous scene end
for mesh in meshes[:-1]:
    plotter.add_mesh(mesh, color="red", show_edges=True, opacity=0.5,)
    plotter.add_points(mesh.points, color="black", opacity=1, render_points_as_spheres=True, point_size=10)
plotter.add_points(meshes[-1].points, color="black", opacity=1, render_points_as_spheres=True, point_size=10)
plotter.write_frame()
plotter.save_graphic(f"{file_name}-pre.pdf")

tetmesh_cells = tetmesh.extract_cells(np.arange(tetmesh.n_cells))
sorted_cell_ids = sort_cell_ids_by_z_x(tetmesh_cells.cell_centers(), np.arange(tetmesh.n_cells))
plotter.add_mesh(tetmesh_cells.extract_cells(60),color="lightgreen", show_edges=True, opacity=1,)
plotter.save_graphic(f"{file_name}-single.pdf")
    

# add all the tetmesh cells one by one 
for i in sorted_cell_ids:
    plotter.add_mesh(tetmesh.extract_cells(i), color="silver", show_edges=True, opacity=1,)
    plotter.write_frame()
    
plotter.save_graphic(f"{file_name}-post.pdf")
plotter.close()


In [9]:
tet_ids_of_cat_cell =[]
for i, cell in enumerate(rel_cells):
    if cell_id in cell.objs:
        tet_ids_of_cat_cell.append(cell.id)
tets_of_cat_cell = tetmesh.extract_cells(tet_ids_of_cat_cell)
tet_ids_of_cat_cell = sort_cell_ids_by_z_x(tets_of_cat_cell.cell_centers(), np.arange(tets_of_cat_cell.n_cells))

sorted_cat_face_ids = sort_cell_ids_by_z_x(cat_meshes[cell_id].cell_centers(), np.arange(cat_meshes[cell_id].n_cells))
# sorted_cat_face_ids.reverse() 

In [10]:
file_name = f"{RES_FOLDER}/4cubes_4_orbit_tets4cell"
plotter = pv.Plotter(off_screen=True)
plotter.open_gif(f"{file_name}.gif",)
#recreate previous scene end
plotter.add_points(meshes[cell_id].points, color="blue", opacity=1, render_points_as_spheres=True, point_size=15)
# plotter.add_mesh(tets_of_cat_cell, color="yellow", show_edges=True, opacity=0.4,)
plotter.add_mesh(meshes[cell_id], color="red", show_edges=True, opacity=0.8,)

plotter.add_mesh(tets_of_cat_cell, color="silver", show_edges=True, opacity=0.0,)
plotter.show()
# plotter.camera.zoom(0.9)


for i, mesh in enumerate(meshes[:-1]):
    plotter.add_mesh(mesh, color="red", show_edges=True, opacity=0.1,)
    plotter.add_points(meshes[i].points, color="black", opacity=0.9, render_points_as_spheres=True, point_size=10)
plotter.add_points(meshes[-1].points, color="black", opacity=0., render_points_as_spheres=True, point_size=10)
# plotter.add_mesh(meshes[-1], color="white", show_edges=True, opacity=0.2,)
# plotter.isometric_view()
plotter.write_frame()
plotter.save_graphic(f"{file_name}-pre.pdf")


for i in tet_ids_of_cat_cell:
    plotter.add_mesh(tets_of_cat_cell.extract_cells(i), color="silver", show_edges=True, opacity=1,)
    plotter.write_frame()

plotter.save_graphic(f"{file_name}-post.pdf")
plotter.close()

Widget(value="<iframe src='http://localhost:63060/index.html?ui=P_0x17ca36110_3&reconnect=auto' style='width: …

In [13]:
file_name = f"{RES_FOLDER}/4cubes_5_orbit_cat"
plotter = pv.Plotter()
plotter.open_gif(f"{file_name}.gif",)
plotter.add_mesh(meshes[cell_id], color="red", show_edges=True, opacity=0.8,)
plotter.add_points(meshes[cell_id].points, color="blue", opacity=1, render_points_as_spheres=True, point_size=15)
tet_actor = plotter.add_mesh(tets_of_cat_cell, color="silver", show_edges=True, opacity=0.4,)
plotter.show()
plotter.save_graphic(f"{file_name}-pre.pdf")

ex_face = plotter.add_mesh(cat_meshes[cell_id].extract_cells(sorted_cat_face_ids[34]), color="yellow", edge_color="purple",show_edges=True, show_vertices=True, line_width=10, opacity=1, vertex_color="purple", render_points_as_spheres=True, point_size=10)
ex_tet = plotter.add_mesh(tets_of_cat_cell.extract_cells(tet_ids_of_cat_cell[13]), color="lightgreen", edge_color="lightgreen",line_width=5, show_edges=True, opacity=0.9,)
ex_tet_e = plotter.add_mesh(tets_of_cat_cell.extract_cells(tet_ids_of_cat_cell[13]).extract_all_edges(), color="lightgreen", line_width=5, show_edges=True, opacity=1,)
i = 2
ex_other = plotter.add_mesh(meshes[i], color="red", show_edges=True, opacity=0.1,)
ex_other_p =plotter.add_points(meshes[i].points, color="red", opacity=1, render_points_as_spheres=True, point_size=10)
plotter.save_graphic(f"{file_name}-single.pdf")
plotter.remove_actor(ex_face)
plotter.remove_actor(ex_tet)
plotter.remove_actor(ex_tet_e) 
plotter.remove_actor(ex_other)
plotter.remove_actor(ex_other_p)
plotter.write_frame()

for i in sorted_cat_face_ids:
    plotter.add_mesh(cat_meshes[cell_id].extract_cells(i), color="yellow", show_edges=True, opacity=1,)
    plotter.write_frame()

plotter.save_graphic(f"{file_name}-post.pdf")
plotter.close()

Widget(value="<iframe src='http://localhost:63060/index.html?ui=P_0x2b76fcd90_4&reconnect=auto' style='width: …

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-13' coro=<RequestHandler.start() done, defined at /opt/homebrew/Caskroom/miniconda/base/envs/irop/lib/python3.10/site-packages/aiohttp/web_protocol.py:462> exception=AssertionError()>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/irop/lib/python3.10/asyncio/tasks.py", line 232, in __step
    result = coro.send(None)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/irop/lib/python3.10/site-packages/aiohttp/web_protocol.py", line 505, in start
    request = self._request_factory(message, payload, self, writer, handler)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/irop/lib/python3.10/site-packages/aiohttp/web_app.py", line 446, in _make_request
    return _cls(
  File "/opt/homebrew/Caskroom/miniconda/base/envs/irop/lib/python3.10/site-packages/aiohttp/web_request.py", line 811, in __init__
    super().__init__(*args, **kwargs)
  File "/opt/homebrew/Caskroom/m

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1987' coro=<WslinkHandler.sendWrappedMessage() done, defined at /opt/homebrew/Caskroom/miniconda/base/envs/irop/lib/python3.10/site-packages/wslink/backends/aiohttp/__init__.py:584> exception=ConnectionResetError('Cannot write to closing transport')>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/irop/lib/python3.10/asyncio/tasks.py", line 232, in __step
    result = coro.send(None)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/irop/lib/python3.10/site-packages/wslink/backends/aiohttp/__init__.py", line 645, in sendWrappedMessage
    await ws.send_str(encMsg)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/irop/lib/python3.10/site-packages/aiohttp/web_ws.py", line 308, in send_str
    await self._writer.send(data, binary=False, compress=compress)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/irop/lib/python3.10/site-packages/aiohttp/http_websocket.py", 

## Sphere inside a sphere

In [12]:

container = pv.Sphere(radius=2).triangulate().clean().rotate_y(1)
sphere = pv.Sphere(radius=1).triangulate().clean()
CDT_DEFAULTS = {
    "nobisect": True,
    "steinerleft": 0,
    "minratio": 10.0,
    "quality": False,
    "cdt": True,
    "switches": "O0/0",
}
meshes = [sphere, container]#, cube5]
tetmesh = compute_cdt(meshes, CDT_DEFAULTS)
# assert(len(tetmesh.points) == len(sphere.points) + len(container.points))
# assert(len(tetmesh.points) == tetmesh.n_points)

steiner_points = tetmesh.points[range(tetmesh.n_points - sum([mesh.n_points for mesh in meshes]))]

# TODO: Filter out steinerpoints. Probably the last points.
print(f'number of steiner points: {len(steiner_points)}')

Delaunizing vertices...
Delaunay seconds:  0.005028

Writing nodes.
Writing elements.
Writing faces.

Output seconds:  0.000233
Total running seconds:  0.005366

Statistics:

  Input points: 1684

  Mesh points: 1684
  Mesh tetrahedra: 8915
  Mesh faces: 18670
  Mesh edges: 11438
  Convex hull faces: 1680

number of steiner points: 0


In [13]:
n_points_per_obj = [mesh.n_points for mesh in meshes]
n_points_per_obj[-1] += len(steiner_points)
face_normals, cat_cells, face_normals_pp = compute_cat_faces(tetmesh, n_points_per_obj, [])
cat_meshes = [pv.PolyData(*convert_faces_to_polydata_input(cell)) for cell in cat_cells]
# FIXME: There are only 2 objects in this situation, however, there are object ids 0,1 and 2
print_cat_analysis(cat_meshes)

cat 0: 3477, is manifold: True
cat 1: 3477, is manifold: True


In [14]:
plotter = pv.Plotter()
plotter.add_mesh(sphere, color="red", show_edges=True, opacity=0.8)
plotter.add_mesh(container, color="white", opacity=0.8)
plotter.add_mesh(cat_meshes[0], color="yellow", show_edges=True, opacity=0.8)
if len(steiner_points) != 0:
    plotter.add_points(steiner_points, color="purple", render_points_as_spheres=True, point_size=10)
plotter.show()


Widget(value="<iframe src='http://localhost:61462/index.html?ui=P_0x176f32c50_3&reconnect=auto' style='width: …

In [15]:
container = pv.Sphere(radius=2).triangulate().clean().rotate_y(1)
sphere_left = pv.Sphere(radius=0.5, center=(-0.6,0,0)).triangulate().clean()
sphere_right = pv.Sphere(radius=0.5, center=(0.6,0,0)).triangulate().clean()
meshes = [sphere_left, sphere_right, container]
tetmesh = compute_cdt(meshes, CDT_DEFAULTS)


steiner_points = tetmesh.points[range(tetmesh.n_points - sum([mesh.n_points for mesh in meshes]))]
print("steiner points: ", len(steiner_points))

n_points_per_obj = [mesh.n_points for mesh in meshes]
n_points_per_obj[-1] += len(steiner_points)
face_normals, cat_cells, face_normals_pp = compute_cat_faces(tetmesh, n_points_per_obj, [])
cat_meshes = [pv.PolyData(*convert_faces_to_polydata_input(cell)) for cell in cat_cells]
print_cat_analysis(cat_meshes)


steiner points:  0
Delaunizing vertices...
Delaunay seconds:  0.009056

Writing nodes.
Writing elements.
Writing faces.

Output seconds:  0.000174
Total running seconds:  0.009294

Statistics:

  Input points: 2526

  Mesh points: 2526
  Mesh tetrahedra: 14806
  Mesh faces: 30452
  Mesh edges: 18171
  Convex hull faces: 1680

cat 0: 3442, is manifold: True
cat 1: 3442, is manifold: True
cat 2: 5108, is manifold: True


In [16]:
plotter= pv.Plotter()
def plot_cat_cells(plotter):
    for mesh in meshes[:-1]:
        plotter.add_mesh(mesh, color="red", show_edges=True, opacity=0.9)
    for mesh in cat_meshes[:-1]:
        plotter.add_mesh(mesh, color="yellow", show_edges=True, opacity=0.7)

    plotter.add_mesh(container, color="white", opacity=0.6)

    if len(steiner_points) != 0:
        plotter.add_points(steiner_points, color="purple", render_points_as_spheres=True, point_size=20)
    plotter.show()

plot_cat_cells(plotter)

plotter= pv.Plotter()
plotter.add_mesh(container, color="white", opacity=0.8)
plotter.add_mesh(cat_meshes[-1], color="green", show_edges=True, opacity=0.8)
plotter.show()

Widget(value="<iframe src='http://localhost:61462/index.html?ui=P_0x176f32e60_4&reconnect=auto' style='width: …

Widget(value="<iframe src='http://localhost:61462/index.html?ui=P_0x176f6c370_5&reconnect=auto' style='width: …

In [17]:
sphere1 = pv.Sphere(radius=0.2, center=(-0.6,0,0), theta_resolution=10, phi_resolution=10).triangulate().clean()
sphere2 = pv.Sphere(radius=0.2, center=(0.6,0,0) , theta_resolution=10, phi_resolution=10).triangulate().clean()
sphere3 = pv.Sphere(radius=0.2, center=(0,0.6,0), theta_resolution=10, phi_resolution=10).triangulate().clean()
sphere4 = pv.Sphere(radius=0.2, center=(0,-0.6,0), theta_resolution=10, phi_resolution=10).triangulate().clean()
sphere5 = pv.Sphere(radius=0.2, center=(0,0,0), theta_resolution=10, phi_resolution=10).triangulate().clean()

meshes = [sphere1, sphere2, sphere3, sphere4, sphere5, container]
tetmesh = compute_cdt(meshes, CDT_DEFAULTS)

steiner_points = tetmesh.points[range(tetmesh.n_points - sum([mesh.n_points for mesh in meshes]))]
print("steiner points: ", len(steiner_points))


n_points_per_obj = [mesh.n_points for mesh in meshes]
n_points_per_obj[-1] += len(steiner_points)
face_normals, cat_cells, face_normals_pp = compute_cat_faces(tetmesh, n_points_per_obj, [])
cat_meshes = [pv.PolyData(*convert_faces_to_polydata_input(cell)) for cell in cat_cells]
print_cat_analysis(cat_meshes)
check_input_and_tetmesh_consistency(tetmesh, meshes)

steiner points:  0
Delaunizing vertices...
Delaunay seconds:  0.00404

Writing nodes.
Writing elements.
Writing faces.

Output seconds:  6.8e-05
Total running seconds:  0.004148

Statistics:

  Input points: 1252

  Mesh points: 1252
  Mesh tetrahedra: 6055
  Mesh faces: 12950
  Mesh edges: 8146
  Convex hull faces: 1680

cat 0: 889, is manifold: True
cat 1: 889, is manifold: True
cat 2: 889, is manifold: True
cat 3: 890, is manifold: True
cat 4: 880, is manifold: True
cat 5: 2517, is manifold: True


In [18]:
plot_cat_cells(pv.Plotter())

plotter = pv.Plotter()
plotter.add_mesh(cat_meshes[-1], color="red", show_edges=True, opacity=0.8)
plotter.show()



Widget(value="<iframe src='http://localhost:61462/index.html?ui=P_0x280330400_6&reconnect=auto' style='width: …

Widget(value="<iframe src='http://localhost:61462/index.html?ui=P_0x176f33b80_7&reconnect=auto' style='width: …

In [19]:
container = pv.Sphere(radius=1, theta_resolution=10, phi_resolution=10).triangulate().clean().rotate_y(1)
sphere1 = pv.Sphere(radius=0.2, center=(-0.6,0,0), theta_resolution=10, phi_resolution=10).triangulate().clean()
sphere2 = pv.Sphere(radius=0.2, center=(0.6,0,0) , theta_resolution=10, phi_resolution=10).triangulate().clean()
sphere3 = pv.Sphere(radius=0.2, center=(0,0.6,0), theta_resolution=10, phi_resolution=10).triangulate().clean()
sphere4 = pv.Sphere(radius=0.2, center=(0,-0.6,0), theta_resolution=10, phi_resolution=10).triangulate().clean()
cube_5 = pv.Cube(center=(0,0,0.6), x_length=0.2, y_length=0.2, z_length=0.2).triangulate().clean()
cube_6 = pv.Cube(center=(0,0,-0.6), x_length=0.2, y_length=0.2, z_length=0.2).triangulate().clean()


meshes = [sphere1, sphere2, sphere3, sphere4, cube_5, cube_6, container]
tetmesh = compute_cdt(meshes, CDT_DEFAULTS)

steiner_points = tetmesh.points[range(tetmesh.n_points - sum([mesh.n_points for mesh in meshes]))]
print("steiner points: ", len(steiner_points))

n_points_per_obj = [mesh.n_points for mesh in meshes]
n_points_per_obj[-1] += len(steiner_points)
face_normals, cat_cells, face_normals_pp = compute_cat_faces(tetmesh, n_points_per_obj, [])
cat_meshes = [pv.PolyData(*convert_faces_to_polydata_input(cell)) for cell in cat_cells]
print_cat_analysis(cat_meshes)

Delaunizing vertices...
steiner points:  0
Delaunay seconds:  0.002012

Writing nodes.
Writing elements.
Writing faces.

Output seconds:  3.4e-05
Total running seconds:  0.00208

Statistics:

  Input points: 426

  Mesh points: 426
  Mesh tetrahedra: 2399
  Mesh faces: 4878
  Mesh edges: 2904
  Convex hull faces: 160

cat 0: 397, is manifold: True
cat 1: 399, is manifold: True
cat 2: 385, is manifold: True
cat 3: 386, is manifold: True
cat 4: 214, is manifold: True
cat 5: 214, is manifold: True
cat 6: 851, is manifold: True


In [20]:
plot_cat_cells(pv.Plotter())

Widget(value="<iframe src='http://localhost:61462/index.html?ui=P_0x28f44b730_8&reconnect=auto' style='width: …

In [21]:
plotter = pv.Plotter()
for mesh in meshes[1:-2]:
    plotter.add_mesh(mesh, color="white", edge_color='gray', opacity=1)
for mesh in cat_meshes[:-1]:
    plotter.add_mesh(mesh, color="yellow", opacity=0.1)

plotter.add_mesh(meshes[-2], color="red",show_edges=True,  opacity=1)
plotter.add_mesh(cat_meshes[-2], color="yellow", show_edges=True, opacity=0.3)
plotter.add_mesh(meshes[0], color="red",show_edges=True,  opacity=1)
plotter.add_mesh(cat_meshes[0], color="yellow", show_edges=True,opacity=0.3)


plotter.add_mesh(container, color="white", opacity=0.2)
plotter.isometric_view()
plotter.save_graphic(f"{RES_FOLDER}/demo-cat-Cubes_and_Spheres.pdf")

plotter.show()

Widget(value="<iframe src='http://localhost:61462/index.html?ui=P_0x2a80aa2c0_9&reconnect=auto' style='width: …

In [22]:
from irregular_object_packing.mesh.transform import scale_and_center_mesh


container = container

rb_cell = pv.read("./../data/mesh/RBC_normal.stl")
rb_cell = scale_and_center_mesh(rb_cell, sphere1.volume)
cell1 = rb_cell.copy().translate((-0.6,0,0))
cell2 = rb_cell.copy().translate((0.6,0,0))
cell3 = rb_cell.copy().translate((0,0.6,0))
cell4 = rb_cell.copy().translate((0,-0.6,0))
cell5 = rb_cell.copy().translate((0,0,0.6))
cell6 = rb_cell.copy().translate((0,0,-0.6))

meshes = [cell1, cell2, cell3, cell4, cell5, cell6, container]

tetmesh = compute_cdt(meshes, CDT_DEFAULTS)
n_points_per_obj = [mesh.n_points for mesh in meshes]
face_normals, cat_cells, face_normals_pp = compute_cat_faces(tetmesh, n_points_per_obj, [])
cat_meshes = [pv.PolyData(*convert_faces_to_polydata_input(cell)) for cell in cat_cells]

Delaunizing vertices...
Delaunay seconds:  0.014718

Writing nodes.
Writing elements.
Writing faces.

Output seconds:  0.000172
Total running seconds:  0.014969

Statistics:

  Input points: 3934

  Mesh points: 3934
  Mesh tetrahedra: 26932
  Mesh faces: 53944
  Mesh edges: 30945
  Convex hull faces: 160



In [ ]:
plot_cat_cells(pv.Plotter())

Widget(value="<iframe src='http://localhost:61462/index.html?ui=P_0x2803680d0_10&reconnect=auto' style='width:…

ERROR:root:Exception raised
ERROR:root:KeyError('ea8ede41c993a99031e096eae605d54d_12d')
ERROR:root:Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/irop/lib/python3.10/site-packages/wslink/backends/aiohttp/__init__.py", line 493, in onMessage
    results = func(*args, **kwargs)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/irop/lib/python3.10/site-packages/trame_vtk/modules/vtk/protocols/local_rendering.py", line 33, in get_array
    self.context.get_cached_data_array(data_hash, binary)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/irop/lib/python3.10/site-packages/trame_vtk/modules/vtk/serializers/synchronization_context.py", line 30, in get_cached_data_array
    cache_obj = self.data_array_cache[p_md5]
KeyError: 'ea8ede41c993a99031e096eae605d54d_12d'

ERROR:root:Exception raised
ERROR:root:KeyError('6667f38a3b5b9245971cf07d9ed5f245_16L')
ERROR:root:Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/irop